# Lab 6: Variable Selection and Regularization

## Part I: Different Model Specs
### A. Regression without regularization
1. Create a pipeline that includes all the columns as predictors for Salary, and performs ordinary linear regression

In [63]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import r2_score, make_scorer, mean_squared_error
from sklearn.compose import ColumnTransformer

In [64]:
hitters = pd.read_csv(r"C:\Users\achur\OneDrive\Desktop\School\CP Fall 2024\544\Hitters.csv")

In [65]:
from sklearn.compose import make_column_selector
def regression_analysis(X, y, model_type = "linear", alpha_values = [0.001, 0.01, 0.1, 1, 10], l1_ratio_values = [0.0, 0.25, 0.5, 0.75, 1.0], cv = 5):
    if model_type == "linear":
        model = LinearRegression()
        alpha = {}
        l1_ratio = {}
    elif model_type == "ridge":
        model = Ridge()
        alpha = {"regression__alpha": alpha_values}
        l1_ratio = {}
    elif model_type == "lasso":
        model = Lasso()
        alpha = {"regression__alpha": alpha_values}
        l1_ratio = {}
    elif model_type == "elastic_net":
        model = ElasticNet()
        alpha = {"regression__alpha": alpha_values, "regression__l1_ratio": l1_ratio_values}
    else:
        raise ValueError("Error")

    # pipeline
    ct = ColumnTransformer([("dummify", OneHotEncoder(sparse_output=False, handle_unknown="ignore"),
                                        make_column_selector(dtype_include=object)),
                            ("standardize", StandardScaler(),
                                        make_column_selector(dtype_include=np.number))],
                                        remainder = "passthrough")

    pipeline = Pipeline([
        ("preprocessing", ct),
        ("regression", model)
    ])

    # grid search
    grid_search = GridSearchCV(pipeline, alpha, cv=cv, scoring="neg_mean_squared_error")
    grid_search_fitted = grid_search.fit(X, y)

    # best model
    best_model = grid_search.best_estimator_
    best_model_fitted = best_model.fit(X, y)

    # coefficients
    coefs = best_model.named_steps["regression"].coef_
    feature_names = best_model_fitted.named_steps["preprocessing"].get_feature_names_out()

    # data frame
    coefs_df = pd.DataFrame({
        "Feature Name": feature_names,
        "Coefficients": coefs})

    print("Cross-validated MSE scores:", -grid_search_fitted.cv_results_["mean_test_score"])

    # best model values
    best_alpha = grid_search.best_params_.get("regression__alpha", None)
    best_l1_ratio = grid_search.best_params_.get("regression__l1_ratio", None)
    best_score = grid_search.best_score_

    # print best alpha and l1 ratio
    if best_alpha is not None:
        print(f"Best alpha: {best_alpha}")
    if best_l1_ratio is not None:
        print(f"Best l1 ratio: {best_l1_ratio}")
    print(f"Best cross-validated MSE score: {-best_score}")

    return coefs_df

2. Fit this pipeline to the full dataset, and interpret a few of the most important coefficients.
3. Use cross-validation to estimate the MSE you would expect if you used this pipeline to predict 1989 salaries

In [69]:
print("----- Linear Regression -----")
coefs_linear = regression_analysis(X, y, model_type="linear")
print(coefs_linear)

----- Linear Regression -----
----- Linear Regression -----


Cross-validated MSE scores: [120548.42849608]
Best cross-validated MSE score: 120548.42849608236
            Feature Name  Coefficients
0      dummify__League_A    -31.299712
1      dummify__League_N     31.299712
2    dummify__Division_E     58.424623
3    dummify__Division_W    -58.424623
4   dummify__NewLeague_A     12.381163
5   dummify__NewLeague_N    -12.381163
6     standardize__AtBat   -291.094556
7      standardize__Hits    337.830479
8     standardize__HmRun     37.853837
9      standardize__Runs    -60.572479
10      standardize__RBI    -26.994984
11    standardize__Walks    135.073897
12    standardize__Years    -16.693359
13   standardize__CAtBat   -391.038655
14    standardize__CHits     86.687617
15   standardize__CHmRun    -14.181723
16    standardize__CRuns    480.747135
17     standardize__CRBI    260.689886
18   standardize__CWalks   -213.892259
19  standardize__PutOuts     78.761296
20  standardize__Assists     53.732490
21   standardize__Errors    -22.160862
Cross-

Interpretation of coefficients: The variable CRuns represents the relationship between total number of runs throughout the career and salary. This means that with each additional career run, the predicted salary is expected to increase by 480.74 units. The variable Hits represents the relationship between the number of hits in a season and salary. This means that with each additional hit in a season, the predicted salary is expected to increase by 337.83 units. The variable CRBI represents the relationship between total number of runs batted in in a career and salary. This means that with every additional RBI in a career, salary is expected to increase by 260.689 units. All expected increases are assuming keeping all other variables constant. 

### B. Ridge regression
1. Create a pipeline that includes all the columns as predictors for Salary, and performs ordinary ridge regression.
2. Use cross-validation to tune the lambda hyperparameter.
3. Fit the pipeline with your chosen lambda to the full dataset, and interpret a few of the most important coefficients.
4. Report the MSE you would expect if you used this pipeline to predict 1989 salaries.

In [67]:
# Ridge Regression (with alpha hyperparameter tuning)
print("\n----- Ridge Regression -----")
coefs_ridge = regression_analysis(X, y, model_type="ridge")
print(coefs_ridge)


----- Ridge Regression -----


----- Ridge Regression -----


Cross-validated MSE scores: [121124.45859214 121022.90328584 120343.62106698 119144.43267692
 119348.9847757 ]
Best alpha: 1
Best cross-validated MSE score: 119144.4326769158
            Feature Name  Coefficients
0      dummify__League_A    -30.438855
1      dummify__League_N     30.438855
2    dummify__Division_E     60.015595
3    dummify__Division_W    -60.015595
4   dummify__NewLeague_A     13.111282
5   dummify__NewLeague_N    -13.111282
6     standardize__AtBat   -270.686441
7      standardize__Hits    296.645050
8     standardize__HmRun     18.100592
9      standardize__Runs    -29.339406
10      standardize__RBI     -9.113295
11    standardize__Walks    124.407173
12    standardize__Years    -38.667748
13   standardize__CAtBat   -225.406548
14    standardize__CHits    126.659607
15   standardize__CHmRun     39.070924
16    standardize__CRuns    320.412169
17     standardize__CRBI    160.386784
18   standardize__CWalks   -184.423611
19  standardize__PutOuts     78.623656
20  st

Interpretation of coefficients: The variable CRuns represents the relationship between total number of runs throughout the career and salary. This means that with each additional career run, the predicted salary is expected to increase by 320.41 units. The variable Hits represents the relationship between the number of hits in a season and salary. This means that with each additional hit in a season, the predicted salary is expected to increase by 296.645 units. The variable CRBI represents the relationship between total number of runs batted in in a career and salary. This means that with every additional RBI in a career, salary is expected to increase by 160.387 units. All expected increases are assuming keeping all other variables constant. 

### C. Lasso Regression
1. Create a pipeline that includes all the columns as predictors for Salary, and performs ordinary ridge regression.
2. Use cross-validation to tune the lambda hyperparameter.
3. Fit the pipeline with your chosen lambda to the full dataset, and interpret a few of the most important coefficients.
4. Report the MSE you would expect if you used this pipeline to predict 1989 salaries.

In [71]:
# Lasso Regression (with alpha hyperparameter tuning)
print("\n----- Lasso Regression -----")
coefs_lasso = regression_analysis(X, y, model_type="lasso")
print(coefs_lasso)


----- Lasso Regression -----


----- Lasso Regression -----


C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.004e+07, tolerance: 4.708e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.984e+06, tolerance: 3.606e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap

C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.537e+06, tolerance: 4.558e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.594e+06, tolerance: 4.708e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap

C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.471e+05, tolerance: 4.281e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.066e+06, tolerance: 4.558e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap

C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.158e+05, tolerance: 4.137e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.929e+04, tolerance: 4.281e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap

C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.815e+03, tolerance: 4.281e+03
  model = cd_fast.enet_coordinate_descent(


Cross-validated MSE scores: [120994.17981481 120964.76468618 120682.25263745 119761.58740741
 121828.14133339]
Best alpha: 1
Best cross-validated MSE score: 119761.58740741303
            Feature Name  Coefficients
0      dummify__League_A -3.582607e+01
1      dummify__League_N  1.734720e-15
2    dummify__Division_E  1.144130e+02
3    dummify__Division_W -2.191014e-11
4   dummify__NewLeague_A  0.000000e+00
5   dummify__NewLeague_N -0.000000e+00
6     standardize__AtBat -2.823710e+02
7      standardize__Hits  3.043595e+02
8     standardize__HmRun  1.112702e+01
9      standardize__Runs -2.496651e+01
10      standardize__RBI -0.000000e+00
11    standardize__Walks  1.206953e+02
12    standardize__Years -3.494815e+01
13   standardize__CAtBat -1.626398e+02
14    standardize__CHits  0.000000e+00
15   standardize__CHmRun  1.422599e+01
16    standardize__CRuns  3.755655e+02
17     standardize__CRBI  1.926109e+02
18   standardize__CWalks -1.896446e+02
19  standardize__PutOuts  7.876037e+01
20  s

Interpretation of coefficients: The variable PutOuts represents the relationship between the total number of put outs in a season and salary. This means that for each additional put out in a season, the expected salary is expected to increase by 7.876 units. The variable Assists represents the relationship between the total number of assists in a season and salary. This means that for each additional assist in a season, the expected salary is expected to increase by 4.1999 units. The variable CRuns represents the relationship between total number of runs in a career and salary. This means that for each additional run, the salary is expected to increase by 3.755 units. All expected increases are assuming keeping all other variables constant. 

### D. Elastic Net
1. Create a pipeline that includes all the columns as predictors for Salary, and performs ordinary ridge regression.
2. Use cross-validation to tune the lambda and alpha hyperparameters.
3. Fit the pipeline with your chosen hyperparameters to the full dataset, and interpret a few of the most important coefficients.
4. Report the MSE you would expect if you used this pipeline to predict 1989 salaries.

In [74]:
# Elastic Net Regression (with alpha and l1_ratio hyperparameter tuning)
print("\n----- Elastic Net Regression -----")
coefs_elastic_net = regression_analysis(X, y, model_type="elastic_net")

# Sort the coefficients in descending order of their absolute values
coefs_elastic_net['Coefficient'] = coefs_elastic_net['Coefficients']
coefs_elastic_net_sorted = coefs_elastic_net.sort_values(by='Coefficient', ascending=False).drop(columns='Coefficient')

print(coefs_elastic_net_sorted)



----- Elastic Net Regression -----


----- Elastic Net Regression -----


C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.045e+07, tolerance: 4.708e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.229e+06, tolerance: 3.606e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in

C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.787e+06, tolerance: 4.558e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.022e+07, tolerance: 4.708e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678

C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.638e+06, tolerance: 4.558e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.012e+07, tolerance: 4.708e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap

C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.579e+06, tolerance: 4.558e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.007e+07, tolerance: 4.708e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap

...

C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.923e+04, tolerance: 4.708e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.726e+05, tolerance: 3.606e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap

C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.147e+07, tolerance: 3.606e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.337e+07, tolerance: 4.137e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in

C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.815e+03, tolerance: 4.281e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.961e+07, tolerance: 4.708e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678

Cross-validated MSE scores: [119911.32888951 120077.76864311 120296.21077347 120590.44872106
 120994.17981481 118957.96789175 119009.79955442 119123.79308625
 119404.65600998 120964.76468618 119805.47261377 119636.17022676
 119381.37557129 119036.41354271 120682.25263745 122029.76100625
 121374.33374621 120775.68540067 120356.59599777 119761.58740741
 150034.24612426 144021.73108229 136766.42534889 128407.50504979
 121828.14133339]
Best alpha: 0.01
Best l1 ratio: 0.0
Best cross-validated MSE score: 118957.96789174949
            Feature Name  Coefficients
7      standardize__Hits    249.932752
16    standardize__CRuns    226.828966
14    standardize__CHits    123.652433
17     standardize__CRBI    122.933345
11    standardize__Walks    111.867856
19  standardize__PutOuts     77.975918
2    dummify__Division_E     60.813166
15   standardize__CHmRun     55.646471
20  standardize__Assists     41.454945
1      dummify__League_N     29.055922
4   dummify__NewLeague_A     12.395109
8     sta

C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.265e+07, tolerance: 5.332e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.265e+07, tolerance: 5.332e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in

Interpretation of coefficients: The variable Hits represents the relationship between total hits in a season and salary. This means that for each additional hit in a season, the salary is expected to increase by 249.93 units. The variable CRuns represents the relationship between total number of runs in a career and salary. This means that with each additional run, the salary is expected to increase by 226.82 units. The variable CHits represents the relationship between total number of hits in a career and salary. This means that with each additional hit in a career, salary is expected to increase by 123.56 units. All expected increases are assuming keeping all other variables constant. 

## Part II. Variable Selection
Based on the above results, decide on:

* Which numeric variable is most important.

* Which five numeric variables are most important

* Which categorical variable is most important

For each of the four model specifications, compare the following possible feature sets:

1. Using only the one best numeric variable.

2. Using only the five best variables.

3. Using the five best numeric variables and their interactions with the one best categorical variable.

Report which combination of features and model performed best, based on the validation metric of MSE.

(Note: lambda and alpha must be re-tuned for each feature set.)

In [90]:
# combine all coefficients into one dataframe
combined_coefs = pd.concat([
    coefs_linear.assign(Model='linear'),
    coefs_ridge.assign(Model='ridge'),
    coefs_lasso.assign(Model='lasso'),
    coefs_elastic_net.assign(Model='elastic_net')
])

# absolute average coefficients for variables
combined_coefs['Absolute Coefficient'] = combined_coefs['Coefficients'].abs()
avg_coefs = combined_coefs.groupby('Feature Name')['Absolute Coefficient'].mean().reset_index()
avg_coefs = avg_coefs.sort_values(by='Absolute Coefficient', ascending=False)

# most important numeric and categorical variables
numeric_vars = avg_coefs[avg_coefs['Feature Name'].str.startswith('standardize__')]
categorical_vars = avg_coefs[avg_coefs['Feature Name'].str.startswith('dummify__')]

# variables again
most_important_numeric = numeric_vars.iloc[0]['Feature Name']
top_five_numeric = numeric_vars.head(5)['Feature Name'].tolist()
most_important_categorical = categorical_vars.iloc[0]['Feature Name']

print(f"Most important numeric variable: {most_important_numeric}")
print(f"Top five numeric variables: {top_five_numeric}")
print(f"Most important categorical variable: {most_important_categorical}")


Most important numeric variable: standardize__CRuns
Top five numeric variables: ['standardize__CRuns', 'standardize__Hits', 'standardize__AtBat', 'standardize__CAtBat', 'standardize__CWalks']
Most important categorical variable: dummify__Division_E
Most important numeric variable: standardize__CRuns
Top five numeric variables: ['standardize__CRuns', 'standardize__Hits', 'standardize__AtBat', 'standardize__CAtBat', 'standardize__CWalks']
Most important categorical variable: dummify__Division_E


In [89]:
# feature sets
# using only the best numeric
X_one_best_numeric = X[['CRuns']]  # Removing 'standardize__' prefix for original data column

# using only the top 5 numeric
top_five_numeric_vars = ['CRuns', 'Hits', 'AtBat', 'CAtBat', 'CWalks']  # Removing 'standardize__' prefix
X_five_best_numeric = X[top_five_numeric_vars]

# 3. using interactions between the five best numeric variables and the most important categorical variable
# one hot encode the most important categorical variable
X_categorical_dummies = pd.get_dummies(X['Division'], drop_first=True)
X_combined = pd.concat([X_five_best_numeric, X_categorical_dummies], axis=1)
poly = PolynomialFeatures(interaction_only=True, include_bias=False)
X_with_interactions = poly.fit_transform(X_combined)

# models and hyperparameters
models = {
    "Linear": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet()
}

param_grids = {
    "Ridge": {"alpha": [0.001, 0.01, 0.1, 1, 10, 100]},
    "Lasso": {"alpha": [0.001, 0.01, 0.1, 1, 10, 100]},
    "ElasticNet": {"alpha": [0.001, 0.01, 0.1, 1, 10, 100], "l1_ratio": [0.1, 0.5, 0.7, 0.9, 1.0]}
}

def evaluate_model(X, y, model, param_grid=None):
    """
    Evaluate the given model using cross-validation.
    If param_grid is provided, perform hyperparameter tuning using GridSearchCV.
    """
    if param_grid:
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
        grid_search.fit(X, y)
        best_model = grid_search.best_estimator_
        best_mse = -grid_search.best_score_
        return best_model, best_mse
    else:
        mse_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
        mean_mse = -np.mean(mse_scores)
        return model, mean_mse

# evaluate feature sets across the four model specifications
feature_sets = {
    "One Best Numeric Variable": X_one_best_numeric,
    "Five Best Numeric Variables": X_five_best_numeric,
    "Five Best Numeric Variables + Interactions": X_with_interactions
}

results = []

for feature_set_name, X_set in feature_sets.items():
    print(f"\n--- Evaluating Feature Set: {feature_set_name} ---")
    for model_name, model in models.items():
        if model_name in param_grids:  # For Ridge, Lasso, ElasticNet
            best_model, mse = evaluate_model(X_set, y, model, param_grids[model_name])
        else:  # For Linear Regression (no tuning)
            best_model, mse = evaluate_model(X_set, y, model)
        
        results.append({
            "Feature Set": feature_set_name,
            "Model": model_name,
            "MSE": mse
        })
        print(f"Model: {model_name}, MSE: {mse:.2f}")

# results
results_df = pd.DataFrame(results)
print("\n--- Comparison Results ---")
print(results_df.sort_values(by="MSE"))



--- Evaluating Feature Set: One Best Numeric Variable ---
Model: Linear, MSE: 143812.94


--- Evaluating Feature Set: One Best Numeric Variable ---
Model: Linear, MSE: 143812.94


Model: Ridge, MSE: 143812.89
Model: Ridge, MSE: 143812.89


Model: Lasso, MSE: 143807.57
Model: Lasso, MSE: 143807.57


Model: ElasticNet, MSE: 143804.49

--- Evaluating Feature Set: Five Best Numeric Variables ---
Model: Linear, MSE: 127649.37
Model: ElasticNet, MSE: 143804.49

--- Evaluating Feature Set: Five Best Numeric Variables ---
Model: Linear, MSE: 127649.37


Model: Ridge, MSE: 127638.17
Model: Ridge, MSE: 127638.17


...

C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.998e+06, tolerance: 4.708e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.389e+06, tolerance: 3.606e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap

C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.274e+06, tolerance: 4.281e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.119e+06, tolerance: 4.558e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap

C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.255e+06, tolerance: 4.708e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.686e+06, tolerance: 3.606e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap

C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.238e+06, tolerance: 4.708e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.682e+06, tolerance: 3.606e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\achur\Downloads\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap

Model: ElasticNet, MSE: 130369.43

--- Comparison Results ---
                                   Feature Set       Model            MSE
7                  Five Best Numeric Variables  ElasticNet  127491.583256
9   Five Best Numeric Variables + Interactions       Ridge  127553.874993
6                  Five Best Numeric Variables       Lasso  127581.856135
5                  Five Best Numeric Variables       Ridge  127638.166078
4                  Five Best Numeric Variables      Linear  127649.370204
8   Five Best Numeric Variables + Interactions      Linear  129296.228072
11  Five Best Numeric Variables + Interactions  ElasticNet  130369.434083
10  Five Best Numeric Variables + Interactions       Lasso  130382.261088
3                    One Best Numeric Variable  ElasticNet  143804.486311
2                    One Best Numeric Variable       Lasso  143807.565325
1                    One Best Numeric Variable       Ridge  143812.894185
0                    One Best Numeric Variable    

## Part III. Discussion

### A. Ridge

Compare your Ridge models with your ordinary regression models. How did your coefficients compare? Why does this make sense?

### B. Lasso

Compare your LASSO model in I with your three LASSO models in II. Did you get the same lambda results? Why does this make sense? Did you get the same MSEs? Why does this make sense?

### C. Elastic Net

Compare your MSEs for the Elastic Net models with those for the Ridge and LASSO models. Why does it make sense that Elastic Net always “wins”?

## Part IV: Final Model

Fit your final best pipeline on the full dataset, and summarize your results in a few short sentences and a plot.

## Appendix and References